# Random Forest

**Project Goal**: To expand the decision tree model into an entire forest (an example of something called an ensemble model); use a grid search cv to tune hyperparameters; and create a function that loads data and a pre-trained model, and uses that model to generate a Series of predictions.

In [4]:
import gzip
import json
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline

#### Import Data

In [5]:
def wrangle(url):
    df = pd.read_csv(url)
    #Re-code the class variable to True = 1, and False = 0:
    df['class'] = df['class'].replace({0: False, 1: True})
    
    #Rename the class variable to bankrupt
    df = df.rename(columns={'class': 'bankrupt'}) #.set_index("year")

    return df

In [6]:
df = wrangle("https://archive.ics.uci.edu/static/public/365/data.csv")
df.head()

C:\Users\Asus\AppData\Local\Temp\ipykernel_14680\3935766308.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = df['class'].replace({0: False, 1: True})


,year,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A56,A57,A58,A59,A60,A61,A62,A63,A64,bankrupt
0,1,0.200550,0.37951,0.39641,2.0472,32.3510,0.38825,0.249760,1.33050,1.1389,...,0.121960,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,False
1,1,0.209120,0.49988,0.47225,1.9447,14.7860,0.00000,0.258340,0.99601,1.6996,...,0.121300,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,False
2,1,0.248660,0.69592,0.26713,1.5548,-1.1523,0.00000,0.309060,0.43695,1.3090,...,0.241140,0.81774,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.2078,False
3,1,0.081483,0.30734,0.45879,2.4928,51.9520,0.14988,0.092704,1.86610,1.0571,...,0.054015,0.14207,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.5497,False
4,1,0.187320,0.61323,0.22960,1.4063,-7.3128,0.18732,0.187320,0.63070,1.1559,...,0.134850,0.48431,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.8980,False


#### Split Data

Split data vertically into X (the features) and y(the target)

In [7]:
target = "bankrupt"
X = df.drop(columns=target)
y = df[target]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (43405, 65)
y shape: (43405,)


Split Data horizontally - train and test sets:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (34724, 65)
y_train shape: (34724,)
X_test shape: (8681, 65)
y_test shape: (8681,)


Resample

In [9]:
over_sampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = over_sampler.fit_resample(X_train, y_train)
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

X_train_over shape: (66098, 65)


,year,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A55,A56,A57,A58,A59,A60,A61,A62,A63,A64
0,4,-0.013224,0.398380,-0.042533,0.83862,-26.922,-0.070108,-0.007444,1.5102,0.710170,...,-2735.00,0.005017,-0.021980,1.01000,0.090420,118.420,3.715600,135.450,2.694600,0.91166
1,4,0.048354,0.396080,0.586460,2.48060,20.699,0.207470,0.072721,1.5247,5.615400,...,554.46,0.002328,0.080067,0.98728,0.000000,22.074,16.626000,25.746,14.177000,321.75000
2,2,-0.452000,0.070426,0.892070,13.66700,393.340,1.043400,-0.452000,13.1990,0.237890,...,5011.40,-0.449260,-0.486250,2.62110,0.000000,NaN,0.838730,108.060,3.377900,6.34300
3,2,-0.020526,0.273260,0.070290,1.26050,17.946,-0.020526,-0.020526,2.5029,0.995670,...,871.98,-0.004346,-0.030012,1.00430,0.004955,44.113,3.207300,101.840,3.583900,1.46580
4,2,-0.170790,0.335700,-0.153930,0.54147,-332.990,-0.170790,-0.170790,1.9185,0.018351,...,-17021.00,-53.493000,-0.265180,54.49300,0.000000,NaN,0.034089,37818.000,0.009652,0.00396


#### Build Model

Baseline Model: Accuracy

In [10]:
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 4))

Baseline Accuracy: 0.9518


- The baseline model predicts a single outcome. The question is whether it is the rigt prediction.

-  If we could somehow use more than one model simultaneously, we'd have a more trustworthy prediction.

- **Ensemble models** work by building multiple models on random subsets of the same data, and then comparing their predictions to make a final prediction. 

- The next step is to create an ensemble of trees here. This type of model is called a **random forest.**

In [11]:
clf = make_pipeline(SimpleImputer(),
                    RandomForestClassifier(random_state=42)
                   )
print(clf)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])
